In [1]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys


import os
import pandas as pd

import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize

import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
import keras.callbacks as kcall
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense
from keras.optimizers import SGD, Adam, rmsprop

%matplotlib inline

/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap

In [2]:
print(os.listdir("data/Reinhard"))

['InSitu', 'Invasive', 'Normal', 'Benign']


In [3]:
## Intilizing variables
output_classes = 4
# batch_size = 32 
# epochs = 30

# sgd_opt = SGD(lr=1E-2, decay=1E-4, momentum=0.9, nesterov=True)
# adam_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1E-4)
# eve_opt = Eve(lr=1E-4, decay=1E-4, beta_1=0.9, beta_2=0.999, beta_3=0.999, small_k=0.1, big_K=10, epsilon=1e-08)

resume_model = False
xception_weights = 'pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [4]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001, 0.000001]),
#          'dropout': hp.choice('dropout', [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]),
         'batch_size': hp.choice('batch_size', [32, 64]),
         'epochs': hp.choice('epochs', [20]),
#        'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'activation': hp.choice('activation',['relu',
#                                                 'tanh']),
        }

In [5]:
train_dir = 'data/Reinhard'
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

data/Reinhard 0
data/Reinhard/InSitu 3500
data/Reinhard/Invasive 3500
data/Reinhard/Normal 3500
data/Reinhard/Benign 3500


In [6]:
def f_nn(params):   
    print ('Parameters testing: ', params)
    batch_size=params['batch_size']
    epochs=params['epochs']

    from keras.preprocessing.image import ImageDataGenerator

    # this is the augmentation configuration we will use for training
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
            width_shift_range=0.2,
            height_shift_range=0.2,
            fill_mode='nearest',
        horizontal_flip=True,
        validation_split=0.25)

    # this is the augmentation configuration we will use for testing:
    # only rescaling
    test_datagen = ImageDataGenerator(rescale=1. / 255)


    #target_size: Tuple of integers (height, width), default: (256, 256). 
    #The dimensions to which all images found will be resized.
    target_size = (256, 256)
    #target_size = (height, width)
    train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size = target_size,       
            class_mode = 'categorical',
            batch_size=32,
            subset="training",
            shuffle = True)

    validation_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size = target_size,        
            class_mode = 'categorical',
            batch_size=32,
            subset = "validation",
            shuffle = True)

    
    model = Sequential()

    model.add(Xception(weights = xception_weights , include_top=False, pooling = 'avg'))
    model.add(Dense(units=output_classes, activation='softmax'))


    model.compile(loss = 'categorical_crossentropy',
                  optimizer = Adam(lr=params['lr'], beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1E-4),
                  metrics = ['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = 2000,
      epochs = epochs,
      validation_data = validation_generator,
#       validation_steps = 100,
      verbose = 1, callbacks=get_callbacks(params))
    
    
    best_epoch = np.argmax(history.history['val_acc'])
    best_val_acc = np.max(history.history['val_acc'])
    print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'val_acc': best_val_acc, 'best_epoch': best_epoch, 'eval_time': time.time(), 'status': STATUS_OK}

In [7]:
output_classes = 4

In [8]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=5, verbose=1),
                ModelCheckpoint('callbacks/{}.h5'.format(params['batch_size']), save_best_only=True),
             TensorBoard('tensor-logs/logs-gridsearch', write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)]
    return callbacks

____

In [ ]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'batch_size': 64, 'epochs': 20, 'lr': 1e-05}
Found 10500 images belonging to 4 classes.
Found 3500 images belonging to 4 classes.
Epoch 1/20
 225/2000 [==>...........................] - ETA: 10:45:24 - loss: 1.2163 - acc: 0.4817